In [1]:
!pip install tensorflow
!pip install matplotlib
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt
from tensorflow import keras
import keras.layers
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,recall_score,precision_score
import numpy as np
import pickle

In [2]:
import zipfile
import os
zip_file_path = './mrismall800.zip'
extract_dir = './mrismall800'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(os.listdir(extract_dir))

['mri']


In [3]:
IMAGE_SIZE=650
BATCH_SIZE=32

In [4]:
import os
print(os.listdir('./mrismall800/mri/dataset'))

['Normal', 'Stroke']


In [5]:
df=tf.keras.preprocessing.image_dataset_from_directory(
    "mrismall800/mri/dataset",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 800 files belonging to 2 classes.


In [6]:
class_names=df.class_names
class_names

['Normal', 'Stroke']

In [7]:
image_count=0
for images, labels in df:
    image_count += images.shape[0]

In [8]:
image_count

800

In [9]:
def  total_images(dataset):
    count = 0
    for _ in dataset:
        count += 1
    return count

In [10]:
def split(df,train_split=0.75,test_split=0.25,shuffle=True,shuffle_size=5000):
  df_shape= total_images(df.unbatch())
  df = df.unbatch().batch(1)
  if shuffle:
    df=df.shuffle(shuffle_size,seed=50)
  train_size=int(train_split*df_shape)
  test_size=int(test_split*df_shape)
  train_df=df.take(train_size)
  test_df=df.skip(train_size).take(test_size)
  return train_df,test_df


In [11]:
train_imgs,test_imgs=split(df)

In [12]:
resize_rescale=tf.keras.Sequential([
    layers.Resizing(650,650),
    layers.Rescaling(1.0/255)
])
data_enhance = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3)
])

In [13]:
def eleven_layer_cnn_softmax_kernel5():
  model = keras.models.Sequential([
      resize_rescale,
    data_enhance,
    keras.layers.Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(650,650,3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=128, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
      ])
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


In [14]:
train_labels = np.concatenate([y.numpy() for _, y in train_imgs], axis=0)
test_labels = np.concatenate([y.numpy() for _, y in test_imgs], axis=0)

In [15]:
model1=eleven_layer_cnn_softmax_kernel5()
model1.fit(train_imgs,epochs=5)
y_pred_probs = model1.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model 1")
print(accuracy)
print(f1)
print(precision)
print(recall)

Epoch 1/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 763s 1s/step - accuracy: 0.7155 - loss: 3.6953
Epoch 2/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 713s 1s/step - accuracy: 0.7639 - loss: 0.6968
Epoch 3/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 671s 1s/step - accuracy: 0.7380 - loss: 0.5849
Epoch 4/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 655s 1s/step - accuracy: 0.7721 - loss: 0.5464
Epoch 5/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 655s 1s/step - accuracy: 0.7492 - loss: 0.5731
200/200 ━━━━━━━━━━━━━━━━━━━━ 95s 465ms/step
Model 1
0.78
0.0
0.0
0.0


In [16]:
def nine_layer_cnn_softmax_kernel5():
  model = keras.models.Sequential([
      resize_rescale,
    data_enhance,
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu', input_shape=(650,650,3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=128, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
      ])
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [17]:
model2=nine_layer_cnn_softmax_kernel5()
model2.fit(train_imgs,epochs=5)
y_pred_probs = model2.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model 2")
print(accuracy)
print(f1)
print(precision)
print(recall)

Epoch 1/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 2417s 4s/step - accuracy: 0.7271 - loss: 2.4080
Epoch 2/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 2481s 4s/step - accuracy: 0.7335 - loss: 0.8252
Epoch 3/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 2572s 4s/step - accuracy: 0.7531 - loss: 0.7906
Epoch 4/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 2349s 4s/step - accuracy: 0.7627 - loss: 0.5615
Epoch 5/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 2340s 4s/step - accuracy: 0.7526 - loss: 0.7047
200/200 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step
Model 2
0.78
0.0
0.0
0.0


In [18]:
def densenet121():
    base_model=tf.keras.applications.DenseNet121(input_shape=(650,650,3),
                                                include_top=False,
                                                weights='imagenet')
    base_model.trainable=False
    model=tf.keras.models.Sequential([
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(512,activation="relu"),
        keras.layers.Dense(2,activation='softmax')
    ])
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

In [19]:
model3=densenet121()
model3.fit(train_imgs,epochs=5)
y_pred_probs = model3.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model")
print(accuracy)
print(f1)
print(precision)
print(recall)

Epoch 1/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 863s 1s/step - accuracy: 0.6752 - loss: 2.3785
Epoch 2/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 823s 1s/step - accuracy: 0.7445 - loss: 0.5561
Epoch 3/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 827s 1s/step - accuracy: 0.7826 - loss: 0.4939
Epoch 4/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 823s 1s/step - accuracy: 0.7910 - loss: 0.4683
Epoch 5/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 821s 1s/step - accuracy: 0.8069 - loss: 0.3873
200/200 ━━━━━━━━━━━━━━━━━━━━ 281s 1s/step
Model
0.74
0.2121212121212121
0.3181818181818182
0.1590909090909091


In [20]:
from sklearn.ensemble import VotingClassifier

In [21]:

def soft_voting(models, X):
    y_pred_probs = [model.predict(X) for model in models]
    avg_pred_probs = np.mean(y_pred_probs, axis=0)
    y_pred = np.argmax(avg_pred_probs, axis=1)
    return y_pred, avg_pred_probs

models = [model1, model2, model3]
y_pred, y_pred_probs = soft_voting(models, test_imgs.map(lambda x, y: x))
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred, average='weighted')
precision = precision_score(test_labels, y_pred, average='weighted')
recall = recall_score(test_labels, y_pred, average='weighted')
print("Model4")
print(accuracy)
print(f1)
print(precision)
print( recall)


200/200 ━━━━━━━━━━━━━━━━━━━━ 97s 476ms/step
200/200 ━━━━━━━━━━━━━━━━━━━━ 262s 1s/step
200/200 ━━━━━━━━━━━━━━━━━━━━ 276s 1s/step
Model4
0.78
0.6835955056179777
0.6084
0.78


In [22]:
test_labels,y_pred_probs

(array([1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0]),
 array([[0.717634  , 0.28236595],
        [0.7030015 , 0.29699856],
        [0.8131327 , 0.18686728],
        [0.7463072 , 0.25369284],
        [0.8234649 , 0.17653513],
        [0.78892916, 0.21107084],
        [0.7613469 , 0.23865312],
        [0.8534322 , 0.14656788],
        [0.8608031 , 0.1391968 ],
        [0.6

In [77]:

def soft_voting(models, X):
    y_pred_probs = [model.predict(X) for model in models]
    avg_pred_probs = np.mean(y_pred_probs, axis=0)    
    print(avg_pred_probs)
    y_pred = []
    for prob in avg_pred_probs:
        if(prob[1]>=0.29):
            y_pred.append(1)
        else:
            y_pred.append(0)
    return np.array(y_pred), avg_pred_probs

In [78]:
def image_preprocess(image_path):
  img=Image.open(image_path).convert('RGB')
  img=img.resize((650,650))
  img=np.array(img)
  img=img/255.0
  return img
def prediction(image_path,location):
  img=image_preprocess(image_path)
  img=np.expand_dims(img,axis=0)
  models = [model1, model2, model3]
  y_pred, y_pred_probs = soft_voting(models, img)
  pred_class=y_pred[0]
  print(f"Final prediction: {y_pred}")
  print(f"Predicted probabilities: {y_pred_probs}") 
#   pred_class=np.argmax(pred[0])
  pred_msg=""
  maps=""
  if pred_class==1:
   pred_msg="Brain Stroke Detected"
   maps_url=f"https://www.google.com/maps/search/hospitals+near+{location}"
   maps=f"Find nearby hospitals: [Google Maps]({maps_url})"   
  else:
   pred_msg= "No brain stroke detected"
   maps="Visiting a Hospital is not required"
    
  return pred_msg,maps
  

In [88]:
img1="58 (9).jpg"
print(prediction(img1,"ashok nagar"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.73448426 0.26551569]]
Final prediction: [0]
Predicted probabilities: [[0.73448426 0.26551569]]
('No brain stroke detected', 'Visiting a Hospital is not required')


In [81]:
!pip install -q gradio
import gradio as gr

In [82]:
interface = gr.Interface(
    fn=prediction,
    inputs=[
gr.Image(type="filepath"),
gr.Textbox(label="Location")],
    outputs=[gr.Textbox(label="Prediction By Model"),
            gr.Markdown(label="Maps")],
    title="Brain Stroke Prediction with  MRI Scans using Ensemble Machine Learning"
)
interface.launch(share=True)



Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://03ae38e28d7311bb65.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.70628434 0.2937156 ]]
Final prediction: [1]
Predicted probabilities: [[0.70628434 0.2937156 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[[0.70628434 0.2937156 ]]
Final prediction: [1]
Predicted probabilities: [[0.70628434 0.2937156 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 698ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[[0.70628434 0.2937156 ]]
Final prediction: [1]
Predicted probabilities: [[0.70628434 0.2937156 ]]
